# Explainer utility in BPMN2CONSTRAINTS

In this notebook, we explore the `Explainer` class, designed to analyze and explain the conformance of traces against predefined constraints. Trace analysis is crucial in domains such as process mining, where understanding the behavior of system executions against expected models can uncover inefficiencies, deviations, or compliance issues.

The constraints currently consists of basic regex, this is because of it's similiarities and likeness to declarative constraints used in BPMN2CONSTRAINTS


## Step 1: Setup

In [14]:
import sys
sys.path.append('../')
from explainer import Explainer, Trace

## Step 2: Basic Usage
Let's start by creating an instance of the `Explainer` and adding a simple constraint that a valid trace should contain the sequence "A" followed by "B" and then "C".


In [15]:
explainer = Explainer()
explainer.add_constraint('A.*B.*C')

## Step 3: Analyzing Trace Conformance

Now, we'll create a trace and check if it conforms to the constraints we've defined.

In [16]:
trace = Trace(['A', 'X', 'B', 'Y', 'C'])
is_conformant = explainer.conformant(trace)
print(f"Is the trace conformant? {is_conformant}")

Is the trace conformant? True


## Step 4: Explaining Non-conformance

If a trace is not conformant, we can use the `minimal_expl` and `counterfactual_expl` methods to understand why and how to adjust the trace.


In [17]:
non_conformant_trace = Trace(['A', 'C'])
print('Constraint: A.*B.*C')
print('Trace:' + str(non_conformant_trace.nodes))
print(explainer.counterfactual_expl(non_conformant_trace)) # Addition
print(explainer.minimal_expl(non_conformant_trace))

non_conformant_trace = Trace(['C', 'B', 'A']) #Reordering
print('-----------')
print('Constraint: A.*B.*C')
print('Trace:' + str(non_conformant_trace.nodes))
print(explainer.counterfactual_expl(non_conformant_trace))
print(explainer.minimal_expl(non_conformant_trace))

non_conformant_trace = Trace(['A','A','C']) #Substitution
print('-----------')
print('Constraint: A.*B.*C')
print('Trace:' + str(non_conformant_trace.nodes))
print(explainer.counterfactual_expl(non_conformant_trace))
print(explainer.minimal_expl(non_conformant_trace))

explainer.remove_constraint(0)
explainer.add_constraint('AC')
non_conformant_trace = Trace(['A', 'X', 'C']) #Substraction
print('-----------')
print('Constraint: AC')
print('Trace:' + str(non_conformant_trace.nodes))
print(explainer.counterfactual_expl(non_conformant_trace))
print(explainer.minimal_expl(non_conformant_trace))




Constraint: A.*B.*C
Trace:['A', 'C']
Non-conformance due to: Constraint (A.*B.*C) is violated by subtrace: ('A', 'C')
-----------
Constraint: A.*B.*C
Trace:['A', 'C']
Suggested change to make the trace (['A', 'C']) conformant: Addition: A->B->C
Non-conformance due to: Constraint (A.*B.*C) is violated by subtrace: ('A', 'C')
-----------
Constraint: A.*B.*C
Trace:['C', 'B', 'A']
Suggested change to make the trace (['C', 'B', 'A']) conformant: Reordering: A->B->C
Non-conformance due to: Constraint (A.*B.*C) is violated by subtrace: ('C', 'B')
-----------
Constraint: A.*B.*C
Trace:['A', 'A', 'C']
Suggested change to make the trace (['A', 'A', 'C']) conformant: Substitution: Replace A with B at position 2
Non-conformance due to: Constraint (A.*B.*C) is violated by subtrace: ('A', 'A')
-----------
Constraint: AC
Trace:['A', 'X', 'C']
Suggested change to make the trace (['A', 'X', 'C']) conformant: Subtraction (Removed X): A->C
Non-conformance due to: Constraint (AC) is violated by subtrace: 

## Step 5: Coming soon